In [31]:
import argparse
from argparse import Namespace
from pathlib import Path
import os
os.environ['CUDA_VISIBLE_DEVICES']='2'
os.environ['HF_HUB_CACHE'] = '/next_share/hf_cache/hub/'
import json
import torch
from transformers import (
    AutoTokenizer, PreTrainedTokenizer, AutoModelForCausalLM
)
from peft import (
    LoraConfig
)
from accelerate import PartialState, Accelerator

import context
os.chdir(context.proj_dir)

from cont_gen.data_loader.cuad_prompt import CUAD_SFT, SFT_Padding, CUAD_SFT_Seq2Seq
from cont_gen.data_loader.cuad_sft import CUAD_SFT_Cached, CUAD_SFT_Filter_Type
from cont_gen.utils.model_utils import build_hf_or_peft_model, smart_resize_embeddings, load_hf_model_from_checkpoint
from cont_gen.trainer.utils import get_smart_optimizer, compute_clm_loss_with_ignore
from cont_gen.trainer.train_only_accelerate import Trainer_Basic, TrainingArgs_Basic
from cont_gen.model.loss import LM_Simple_Feed
from cont_gen.run.infer_sft import SimpleGenerator, load_test_dataset
from cont_gen.data_process.utils import tokenize_wo_bos

In [2]:
def build_tkn(path):
    return AutoTokenizer.from_pretrained(path, trust_remote_code = True)

# tokenizer name to path
TKN_MAP = {
    'flan-t5': build_tkn('google/flan-t5-large'),
    'llama2': build_tkn('meta-llama/Llama-2-7b-hf'),
    'llama3': build_tkn('meta-llama/Meta-Llama-3-8B'),
    'mistral': build_tkn('mistralai/Mistral-7B-v0.1'),
    # 'phi1': build_tkn('microsoft/phi-1_5'),
    'phi2': build_tkn('microsoft/phi-2')
}

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [22]:
tk_name='llama3'
model_path = 'meta-llama/Meta-Llama-3-8B-Instruct'
is_seq2seq = False
ckpt = 'runs/ood/llama3_chat/seed42_tr29/pmt_01_all_lr1e-5_bs16_wd0.0/checkpoint-15692'

data_path = f'data/ood_split/seed42_tr29/{tk_name}/pmt_01/train_data.jsonl'
# tokenizer = TKN_MAP[tk_name]
tokenizer = build_tkn(model_path)

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [25]:
print(tokenizer.special_tokens_map)

{'bos_token': '<|begin_of_text|>', 'eos_token': '<|eot_id|>'}


In [33]:
tokenizer.convert_ids_to_tokens(tokenize_wo_bos(tokenizer, 'Hello world.').input_ids)

['Hello', 'Ġworld', '.']

In [30]:
tokenizer.convert_tokens_to_ids(['<|eot_id|>'])

[128009]

In [36]:
tokenizer.name_or_path

'meta-llama/Meta-Llama-3-8B-Instruct'

: 

In [7]:
tk_name='mistral'
model_path = 'mistralai/Mistral-7B-v0.1'
is_seq2seq = False
# ckpt = 'runs/ood/llama3/seed42_tr29/pmt_01_lr1e-5_bs16_wd0.0/checkpoint-15692'

data_path = f'data/ood_split/seed42_tr29/{tk_name}/pmt_01/train_data.jsonl'
tokenizer = TKN_MAP[tk_name]

In [4]:
# Load dataset
tr_ds = CUAD_SFT_Cached(data_path, tokenizer, is_seq2seq = is_seq2seq,
                        is_chat = True, 
                          cache_dir = Path(data_path).parent / 'cache')

Load from cache: data/ood_split/seed42_tr29/llama3/pmt_01/cache/cached_train_data.jsonl_Meta-Llama-3-8B-Instruct_v1.1chat.pkl


In [5]:
if 'model' in dir():
    del model
model = AutoModelForCausalLM.from_pretrained(model_path, torch_dtype = torch.bfloat16, device_map = 0)

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

In [6]:
adp_st = torch.load(Path(ckpt) / 'adapter_model.bin', map_location='cpu')
print([k for k in adp_st.keys() if 'layers' not in k])

[]


In [14]:
if 'base_model.model.model.embed_tokens.weight' in adp_st:
    new_shape = adp_st['base_model.model.model.embed_tokens.weight'].shape[0]
    model.resize_token_embeddings(new_shape)
model.load_adapter(ckpt)

In [7]:
def to_cuda(data):
    return {k:v.cuda() for k,v in data.items()}

def to_batch(data):
    return {k:torch.tensor(v).unsqueeze(0).cuda() for k,v in data.items()}

def add_target_head(sample, head):
    """Append target head to source"""
    ori_ids = sample['input_ids']
    ipt_len = len([k for k in sample['labels'] if k == -100])
    new_ids = ori_ids[:ipt_len] + list(head)
    new_mask = [1] * len(new_ids)
    new_labels = [-100] * ipt_len + list(head)
    return {'input_ids': new_ids, 'attention_mask': new_mask, 'labels': new_labels}

def generate(model, batch, tokenizer, max_len = 512):
    return model.generate(**{k:batch[k] for k in ['input_ids', 'attention_mask']},
                          do_sample = False, eos_token_id = tokenizer.eos_token_id,
                          max_new_tokens = max_len)

def get_prob(logits, top_k = 5):
    probs= torch.softmax(logits, dim = -1)
    rank = torch.argsort(probs, descending = True).tolist()
    return [(rank[i], probs[rank[i]].item()) for i in range(top_k)]

def get_token_prob(logits, tokenizer, top_k = 5):
    top = get_prob(logits, top_k)
    return [(tokenizer.convert_ids_to_tokens(tid), tid, p) for tid, p in top]

def greedy_generate(model, batch, tokenizer, num_token = 5):
    past_key_values = []
    input_ids = batch['input_ids']
    mask = batch['attention_mask']
    gen_tokens = []
    for step in range(num_token):
        with torch.no_grad():
            out = model(input_ids = input_ids, attention_mask = mask, past_key_values = past_key_values)
        top = get_token_prob(out.logits[0][-1], tokenizer)
        print(f'Step {step+1}')
        for token, tid, p in top:
            print(f'\t{token} {tid} {p:.4f}')
        past_key_values = out.past_key_values
        input_ids = torch.tensor([[top[0][1]]]).cuda()
        mask = torch.concat([mask, torch.tensor([[1]]).cuda()], dim = -1)
        gen_tokens.append(top[0][1])
    return gen_tokens

In [15]:
sample = tr_ds[0]

prompt = add_target_head(sample, [])
pmt_len = len(prompt['input_ids'])
with torch.no_grad():
    ba = to_batch(prompt)
    out = model(**ba)
    gen_out = generate(model, ba, tokenizer, 20)

/storage_fast/rhshui/lib/anaconda3/envs/llm/lib/python3.9/site-packages/transformers/generation/configuration_utils.py:389: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.6` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
/storage_fast/rhshui/lib/anaconda3/envs/llm/lib/python3.9/site-packages/transformers/generation/configuration_utils.py:394: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.9` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`.
  warnings.warn(
Setting `pad_token_id` to `eos_token_id`:128009 for open-end generation.


In [20]:
tokenizer.decode(sample['input_ids'][-100:])

' an energy efficiency device, which is referred to as an "Energy Saver" which may be improved or otherwise changed from its present composition (the "Products"). The Company may engage in the business of selling other products or other devices other than the Products, which will be considered Products if Distributor exercises its options pursuant to Section 7 hereof.\n\n###Question: The date of the contract\n\n###Answer:<|eot_id|><|start_header_id|>assistant<|end_header_id|>\n\n- 7th day of September, 1999.<|end_of_text|>'

In [21]:
tokenizer.special_tokens_map

{'bos_token': '<|begin_of_text|>', 'eos_token': '<|eot_id|>'}

In [16]:
greedy_generate(model, to_batch(prompt), tokenizer)

Step 1
	- 12 0.8060
	No 2822 0.1401
	7 22 0.0313
	This 2028 0.0062
	this 576 0.0054
Step 2
	Ġthis 420 0.7154
	Ġ 220 0.2050
	ĠSeptember 6250 0.0518
	ĠThe 578 0.0102
	ĠThis 1115 0.0079
Step 3
	Ġ 220 0.9996
	<|end_of_text|> 128001 0.0002
	Ċ 198 0.0001
	Ġ? 949 0.0000
	? 30 0.0000
Step 4
	7 22 1.0000
	Ġ 220 0.0000
	Ġseventh 31487 0.0000
	6 21 0.0000
	17 1114 0.0000
Step 5
	th 339 0.9999
	< 27 0.0000
	(th 25364 0.0000
	^ 61 0.0000
	Ġth 270 0.0000


[12, 420, 220, 22, 339]

In [17]:
print(out.keys())
print(len(prompt['input_ids']))
out.logits[0][-1].shape

odict_keys(['loss', 'logits', 'past_key_values'])
205


torch.Size([128256])

In [18]:
gen_tokens = gen_out[0][pmt_len:]
print(gen_tokens)
print(tokenizer.convert_ids_to_tokens(gen_tokens))

tensor([    12,    420,    220,     22,    339,   1938,    315,   6250,     11,
           220,   2550,     24,     13, 128001, 128006, 128006,  78191, 128007,
           271,     40], device='cuda:0')
['-', 'Ġthis', 'Ġ', '7', 'th', 'Ġday', 'Ġof', 'ĠSeptember', ',', 'Ġ', '199', '9', '.', '<|end_of_text|>', '<|start_header_id|>', '<|start_header_id|>', 'assistant', '<|end_header_id|>', 'ĊĊ', 'I']


In [19]:
tokenizer.eos_token

'<|eot_id|>'

In [48]:
get_token_prob(out.logits[0][-1], tokenizer)

[('<0x0A>', 13, 0.4116723835468292),
 ('▁', 28705, 0.2496919333934784),
 ('▁September', 4074, 0.10408708453178406),
 ('▁The', 415, 0.0593070350587368),
 ('▁"', 345, 0.026317333802580833)]

In [12]:
print(tokenizer.decode(sample['input_ids'][-50:]))

considered Products if Distributor exercises its options pursuant to Section 7 hereof.

###Question: The date of the contract

###Answer: - 7th day of September, 1999.</s>


In [36]:
TKN_MAP['phi2'].convert_ids_to_tokens(TKN_MAP['phi2'].encode('hello world'))

['hello', 'Ġworld']

In [16]:
tokenizer.bos_token_id

128000

In [24]:
vec = model.model.embed_tokens.weight[128:135]

In [25]:
torch.norm(vec, dim = 1)

tensor([0.6055, 0.6016, 0.6289, 0.6289, 0.6289, 0.6055, 0.6211],
       device='cuda:0', dtype=torch.bfloat16,
       grad_fn=<LinalgVectorNormBackward0>)

In [21]:
torch.norm(model.model.embed_tokens.weight, dim = 1).mean()

tensor(0.5938, device='cuda:0', dtype=torch.bfloat16, grad_fn=<MeanBackward0>)

In [4]:
test_ds = CUAD_SFT_Filter_Type(
        'data/ood_split/seed42_tr29/llama3/pmt_01/test_data_ood.jsonl',
        tokenizer,
        is_seq2seq = is_seq2seq,
        is_chat = True,
        # cache_dir = Path(args.data_path).parent / 'cache',
        is_test = True,
        judge_type_fn = lambda k: k>0,
    )

100%|███████████████████████████████████| 11808/11808 [00:06<00:00, 1753.68it/s]


In [5]:
print(tokenizer.decode(test_ds[0]['input_ids']))

<|begin_of_text|><|start_header_id|>user<|end_header_id|>

You are a helpful assistant. Review the contract clauses and answer questions. Output the mentioned clauses if exist; otherwise output "No".

###Clauses:
Exhibit 10.16 SUPPLY CONTRACT Contract No: Date: The buyer/End-User: Shenzhen LOHAS Supply Chain Management Co., Ltd. ADD: Tel No. : Fax No. : The seller: ADD: The Contract is concluded and signed by the Buyer and Seller on, in Hong Kong. 1. General provisions 1.1 This is a framework agreement, the terms and conditions are applied to all purchase orders which signed by this agreement (hereinafter referred to as the "order"). 1.2 If the provisions of the agreement are inconsistent with the order, the order shall prevail. Not stated in order content will be subject to the provisions of agreement. Any modification, supplementary, give up should been written records, only to be valid by buyers and sellers authorized representative signature and confirmation, otherwise will be deem